# Title

#### Explain

Importamos librerías

In [28]:
from pymongo import MongoClient
import pandas as pd
from src.gameslib import (geocode,gMapsLink,toCartoPoint,toGeoJSON)
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()
import re

Indicamos la base de datos de MongoDB (en carpeta /output, unzip y cargar en servidor local mongodb)

In [29]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [40]:
empresas = list(db["cleancompanies"].find({}))
df = pd.DataFrame(empresas)
df.groupby(['category_code']).size().nlargest(10)

category_code
web                2307
software           1885
mobile              682
advertising         679
games_video         644
other               625
enterprise          542
consulting          445
ecommerce           412
network_hosting     410
dtype: int64

buscamos las empresas de videojuegos, diseño, etc

In [30]:

query = {"$or": [{"category_code":{'$regex':'/advertising|web|games_video|software/'}},{"tag_list":{'$regex':'/design|game|e-sport/'}}]}

designers = list(db["cleancompanies"].find(query))


In [31]:

df = pd.DataFrame(designers)
df.head()

,_id,name,category_code,number_of_employees,tag_list,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,geometry,location
0,5e413a48603a43834ac1351d,Lala,games_video,5.0,,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369,POINT (-122.154369 37.451151),"{'type': 'Point', 'coordinates': [-122.154369,..."
1,5e413a48603a43834ac1351e,Flektor,games_video,5.0,"flektor, photo, video",None,"8536 National Blvd, Suite A",None,90232,Culver City,CA,USA,34.025958,-118.379768,POINT (-118.379768 34.025958),"{'type': 'Point', 'coordinates': [-118.379768,..."
2,5e413a48603a43834ac13524,Joost,games_video,0.0,"iptv, babelgum, television, video, thevenicepr...",,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,40.746497,-74.009447,POINT (-74.0094471 40.7464969),"{'type': 'Point', 'coordinates': [-74.0094471,..."
3,5e413a48603a43834ac13526,Babelgum,games_video,5.0,"iptv, web2ireland",,,,,London,None,GBR,53.344104,-6.267494,POINT (-6.267494 53.344104),"{'type': 'Point', 'coordinates': [-6.267494, 5..."
4,5e413a48603a43834ac13532,Sparter,games_video,5.0,"gaming, game, wow, worldofwarcraft, virtualgoods",None,None,None,None,None,None,USA,37.090240,-95.712891,POINT (-95.712891 37.09024),"{'type': 'Point', 'coordinates': [-95.712891, ..."


In [13]:
import geopandas as gpd

from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import size_continuous_layer
from cartoframes.viz.widgets import histogram_widget

In [14]:
Map(Layer(designers))

In [15]:
Map(size_continuous_layer(cleanData, 'number_of_employees'))

In [ ]:
'''import math
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None
        

cleanData["location"] = cleanData[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
cleanData[["latitude","longitude","location"]]
'''

In [16]:

def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }

ciudad = geocode("Köln")
ciudad

{'type': 'Point', 'coordinates': [6.9665, 50.93545]}

In [6]:
lat='38.345887'
long='-0.490576'
tipositio='school'
radio='2500'
API_KEY = os.getenv("PLACES_API_TOKEN")
URL = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{long}&type={tipositio}&radius={radio}&key={API_KEY}'


In [25]:
response_data = requests.get(URL)


In [8]:
data = len(list(response_data))
data

254

In [26]:
json_data = json.loads(response_data.text)
lista_coles = json_data['results']
for i in range(len(lista_coles)):
    print(lista_coles[i]['geometry']['location'])

{'lat': 38.3567087, 'lng': -0.5058289}
{'lat': 38.34958, 'lng': -0.4740800000000001}
{'lat': 38.3523635, 'lng': -0.5051052}
{'lat': 38.34316400000001, 'lng': -0.492161}
{'lat': 38.3619991, 'lng': -0.4909873000000001}
{'lat': 38.3534778, 'lng': -0.4856677}
{'lat': 38.3466503, 'lng': -0.5062078999999999}
{'lat': 38.3392515, 'lng': -0.5077184}
{'lat': 38.3553565, 'lng': -0.4739583999999999}
{'lat': 38.3447153, 'lng': -0.4900758}
{'lat': 38.345251, 'lng': -0.491248}
{'lat': 38.3411174, 'lng': -0.4958768}
{'lat': 38.3432417, 'lng': -0.4840492}
{'lat': 38.35493140000001, 'lng': -0.4869952999999999}
{'lat': 38.3400287, 'lng': -0.4991959}
{'lat': 38.35236520000001, 'lng': -0.5032013}
{'lat': 38.3534297, 'lng': -0.4964602}
{'lat': 38.34400730000001, 'lng': -0.4859191}
{'lat': 38.35245219999999, 'lng': -0.4748248999999999}
{'lat': 38.3486923, 'lng': -0.4747926}


In [15]:
# From Response/JSON to dict:
# data_dict = json.loads(response_data.text)
# From dict to json:
# y = json.dumps(data_dict)



In [46]:
lat='38.345887'
long='-0.490576'
radio='1000'
placename = 'Starbucks'
URL = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={placename}&inputtype=textquery&fields=formatted_address,name&locationbias=circle:{radio}@{lat},{long}&key={API_KEY}'

In [47]:
response_data = requests.get(URL)


In [48]:
json_data = json.loads(response_data.text)
#starb = json_data['results']
json_data

{'candidates': [{'formatted_address': 'Av. Maisonnave, 53, 03003 Alicante, España',
   'name': 'Starbucks'}],
 'status': 'OK'}